<a href="https://colab.research.google.com/github/angel-araiza/first-ML-Model/blob/main/first_ml_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.0


loading data here:



In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Building a machine learning model:
Build a tf.keras.Sequential model

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation = 'relu'),
    # The dense layer means that all the previous inputs are connected
    # to the next layer of all the inputs
    tf.keras.layers.Dropout(0.2), # What does Dropout mean?
    # The dropout layer, randomly sets up input units to 0, with a frequency
    # rate as eash step during the - TRAINING- time, this helps prevent
    # over fitting. Inputs not set to 0, are scaled up by "1 / (1-rate)" such
    # that the sum over all the inputs is unchanged
    tf.keras.layers.Dense(10)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Sequential is class that is useful when stacking layers (when each layer has 1 input tensor and 1 output tensor) .       * Most tf models are composed of layers.      *
* This model returns a vector of logits or log-odds scores, one for each class [What are logits or log-odds?]
- "logits" are the unprocessed prediction scores of that your model outputs (from the second to last layer of the model)
- "log-odds"  - is the logorithm of the probability of the event happening or not.

In [11]:
predictions = model(x_train[:1]).numpy() # what does ":1" mean?
#is that the slice of just the first item in the array?
predictions

array([[-0.04933465, -0.13350551,  0.6854698 ,  0.0279381 , -0.41689253,
        -0.29725966, -0.4768116 , -0.27932826,  0.2960616 , -0.3170558 ]],
      dtype=float32)

The "tf.nn.softmax" function converts these logits to probabilities for each class:

In [12]:
tf.nn.softmax(predictions).numpy()

array([[0.09821361, 0.09028523, 0.20478266, 0.10610376, 0.06800538,
        0.07664771, 0.06405024, 0.07803451, 0.13873158, 0.0751453 ]],
      dtype=float32)

This is our loss function for training. And this  vvv is how you define it:

losses.SparseCategoricalCrossentropy


In [13]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


The loss function takes the vector of the ground truth values and a vector of logits and returns a scalar loss for each example. - What is a scalar loss?
* The loss is equal to the negative log probability of the true class: meaning the loss is zero if the model is sure of the correct class.
* _btw_ the scalar loss  is one number that represents the error of a models predications comared to the actual target values during training. (it measures how well the model is performing)

In [14]:
loss_fn(y_train[:1], predictions).numpy()

2.5685353

Before you start training your model you need to:
1. Set-up the model
2. compile the model (using Keras -> Model.compile)
- Set the optimizer class to "adam"
- Set the loss to the loss_fn
- Set what metric you want the model to be evaluated by (we will set metrics parameter to "accuracy"


In [15]:
model.compile(optimizer = 'adam',
              loss=loss_fn,
              metrics=['accuracy'])

 \- Train and evaluate your model -
Use the "Model.fit" method to adjust your model parameters and minimize the loss

In [16]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8589 - loss: 0.4841
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9565 - loss: 0.1491
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9688 - loss: 0.1052
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9744 - loss: 0.0845
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9772 - loss: 0.0744


The "Model.evaluate" method checks the model's performance, usually on a validation set or test set.

In [17]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - 2ms/step - accuracy: 0.9775 - loss: 0.0734


[0.0734427198767662, 0.9775000214576721]

In [18]:
# if you want your model to return a probability
# you can wrap the trained model, and attach the softmax to it:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.32123796e-08, 2.72963536e-08, 3.06903530e-06, 4.91089668e-05,
        1.95364298e-11, 1.70647194e-08, 1.13830806e-13, 9.99945402e-01,
        2.01513174e-07, 2.22623248e-06],
       [4.33173170e-07, 7.38193703e-05, 9.99833107e-01, 9.15482815e-05,
        1.23323643e-14, 1.97725129e-07, 4.12798045e-08, 1.17103105e-13,
        8.27078395e-07, 8.19508171e-14],
       [1.76895014e-07, 9.98900056e-01, 1.70116346e-05, 4.06348590e-06,
        3.12762022e-05, 2.52439859e-05, 4.99997577e-06, 9.53756215e-04,
        5.98882034e-05, 3.40678753e-06],
       [9.99951720e-01, 3.83431553e-09, 1.41687569e-05, 2.77388551e-07,
        1.70383800e-07, 7.20243224e-06, 4.27232499e-06, 7.68135942e-06,
        2.97496978e-08, 1.45519225e-05],
       [7.44751651e-06, 1.09636726e-08, 4.78238981e-05, 2.75613843e-06,
        9.83906627e-01, 7.73152897e-06, 5.98952874e-06, 1.36830975e-04,
        6.03289118e-06, 1.58787649e-02]], dtype=float32)>